## Network Validation Checks

#### Import Libraries 

In [ ]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
# from nxviz import CircosPlot

#### Load Network data from Shapefiles

In [ ]:
line_shp_load = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master.shp')).to_crs(epsg=2274)
node_shp_load = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master_Node.shp')).to_crs(epsg=2274)

### Build Network Graph

#### Turn Two-Way Links into 2 One-Way Links for Directional Flow.
#### Create Function: *directional_links*
Keep AB Links and produce BA Links, carrying over link attributes based on the direction of flow.

In [ ]:
def directional_links(atts, line_shp):
    dirs = {'keep':'AB', 'invert':'BA'}
    dfs = {}
    
    for flow,d in dirs.items():
        links = line_shp[line_shp['{}_LANES'.format(d)] > 0].copy()
        rename_dict = {'{}{}'.format(d, a):a for a in atts}
        links = links.rename(columns=rename_dict)
        
        if flow == 'invert':
            links['A'] = links['TO_ID']
            links['B'] = links['FROM_ID']
            links = links.drop(columns=['FROM_ID', 'TO_ID']).rename(columns={'A': 'FROM_ID', 'B': 'TO_ID'})
        
        dfs['{}_links'.format(d)] = links
    
    links = pd.concat(dfs.values(), ignore_index=True)
    
    drop_dict = ['{}{}'.format(d, a) for a in atts for d in dirs.values()]
    links = links.drop(columns=drop_dict)
    
    return links

#### Generate Directional links with Attributes
####  Call Function: *directional_links*
Use list of Directional Attributes to carry over, e.g. 'AB_LANES' and 'BA_LANES', to '_LANES' 

In [ ]:
atts = [
    '_LINKDIR',
    '_LINKDTR',
    '_LANES',
    '_PARKING',
    '_TRIMS',
    '_AADT',
    '_CAR_ADT',
    '_SUT_ADT',
    '_MUT_ADT',
    '_BASEVOL',
    '_AFFTIME',
    '_AFFSPD',
    '_UCDELAY',
    '_DLYCAP',
    '_AMCAP',
    '_PMCAP',
    '_BPRA',
    '_BPRB'
]

line_shp = directional_links(atts, line_shp_load)

####  Build the Directional Graph: *g*
#### Call Method: *nx.from_pandas_edgelist*

In [ ]:
g = nx.from_pandas_edgelist(
    line_shp,
    source='FROM_ID',
    target='TO_ID',
    edge_attr=True,
    create_using=nx.DiGraph()
)

assert nx.is_directed(g)

### Link to Link validation

#### Create Function: *link_matcher*: 
For each node in Network, match all combinations of In/ Out Links and return difference (Substraction)

In [ ]:
def link_matcher(*atts, graph, edge_id='ID'):
    node_feed = []

    for node in node_shp_load['ID']:
        if node not in g:
            print('Node not in network: {}'.format(node))
            continue

        for i_o, i_d, i_data  in g.in_edges(node, data=True):
            for o_o, o_d, o_data in g.out_edges(node, data=True):
                if i_o == o_d and i_d == o_o:
                    # This is just a U-Turn on the "same" link
                    continue
                    
                data = [i_data, o_data]                
                node_feed.append([node, *[d[edge_id] for d in data], *[o_data[a] - i_data[a] for a in atts]])

    # Generate Column names
    dirs = ['in', 'out']
    cols = ['node', *[ d+'_'+edge_id for d in dirs], *['diff_'+a for a in atts]]

    return pd.DataFrame(node_feed, columns=cols)

#### Create Function: *attribute_filter*: 
Apply Attribute value to filter 

In [ ]:
def attribute_filter(network_att_chg, att_name, att_filter_val, get_node_max):
    fld_diff = 'diff_{}'.format(att_name)
    abs_fld_diff = 'abs_diff_{}'.format(att_name)

    network_att_chg = network_att_chg[abs(network_att_chg[fld_diff])>=att_filter_val].reset_index(drop=True)
    network_att_chg[abs_fld_diff] = abs(network_att_chg[fld_diff])

    if get_node_max:
        network_att_chg['sorted'] = network_att_chg.sort_values([abs_fld_diff]).groupby('node').cumcount()+1
        network_att_chg = network_att_chg[network_att_chg['sorted']==1][['node', 'in_ID', 'out_ID', fld_diff]]

    return network_att_chg.sort_values(by=fld_diff, ascending=False)

#### Create Function: *export_geo*: 
Export Link To/From values, from Node Maximum

In [ ]:
# In/Out Long Table
def export_geo(network_speed_chg, att_name):
    fld_diff = 'diff_{}'.format(att_name)
    
    in_df = network_speed_chg[['node', 'in_ID']].copy()
    in_df.columns = ['node', 'line_ID']

    network_speed_chg = network_speed_chg[['node', 'out_ID', fld_diff]].copy()
    network_speed_chg.columns = ['node', 'line_ID', fld_diff]

    network_speed_chg = network_speed_chg.append(in_df)

    # geometries
    network_speed_chg = pd.merge(line_shp_load[['ID', 'geometry']], network_speed_chg, left_on='ID', right_on='line_ID')
    
    network_speed_chg.to_file(os.path.join('..', 'data', 'network_speed_change_{}.shp'.format(fld_diff)))
    return network_speed_chg

### Network Link-to-Link attribute validation

#### Call Function: *network_matcher*
#### Call Function: *attribute_filter*

In [ ]:
# Specify Parameters
# Attribute to validate:
attribute_name = 'SPD_LMT'
#Equal to or Greater than:
attribute_filter_value = 20 
#Summarize by Node Maximum value and Export Geo:
get_node_max = True


network_att_change = link_matcher(attribute_name, graph=g)
network_att_change = attribute_filter(network_att_change, attribute_name, attribute_filter_value, get_node_max)
if get_node_max:
    export_geo(network_att_change, attribute_name)

# Print
print('Total records above specified value: {:,}'.format(len(network_att_change)))
network_att_change.head()

In [ ]:
### TO DO

#### Keep Inputs

In [ ]:
def network_matcher_with_inputs(*atts, graph, edge_id='ID'):
    node_feed = []

    for node in node_shp_load['ID']:
        if node not in g:
            print('Node not in network: {}'.format(node))
            continue

        for i_o, i_d, i_data  in g.in_edges(node, data=True):
            for o_o, o_d, o_data in g.out_edges(node, data=True):
                if i_o == o_d and i_d == o_o:
                    # This is just a U-Turn on the "same" link
                    continue
                    
                data = [i_data, o_data]                
                node_feed.append([node, *[d[edge_id] for d in data], *[d[a] for d in data for a in atts]])

    # Generate Column names
    dirs = ['in', 'out']
    cols = ['node', *[ d+'_'+edge_id for d in dirs], *[ d+'_'+a for d in dirs for a in atts]]

    return pd.DataFrame(node_feed, columns=cols)

In [ ]:
network_speeds = network_matcher_with_inputs('SPD_LMT', graph=g)

network_speeds.head(5)

In [ ]:
#FIX
# Function Class
network_speeds = network_matcher_with_inputs('SPD_LMT', 'FUNC_CLASS', 'RTE_NME', graph=g)
network_speeds = network_speeds[abs(network_speeds['out_SPD_LMT'] - network_speeds['in_SPD_LMT']) > 10]
network_speeds = network_speeds[['in_FUNC_CLASS', 'out_FUNC_CLASS', 'in_SPD_LMT', 'out_SPD_LMT']].reset_index()

network_speeds = network_speeds[['in_FUNC_CLASS', 'out_FUNC_CLASS', 'in_SPD_LMT', 'out_SPD_LMT']].reset_index()
network_speeds.groupby(['in_FUNC_CLASS', 'out_FUNC_CLASS', 'in_SPD_LMT', 'out_SPD_LMT']).count().reset_index()

In [ ]:
# Route Name
network_speeds = network_matcher_with_inputs('SPD_LMT', 'FUNC_CLASS', 'RTE_NME', graph=g)
network_speeds = network_speeds[abs(network_speeds['out_SPD_LMT'] - network_speeds['in_SPD_LMT']) > 10]
network_speeds = network_speeds[['in_RTE_NME', 'out_RTE_NME', 'in_SPD_LMT', 'out_SPD_LMT']].reset_index()

print(len(network_speeds))
network_speeds[(network_speeds['in_RTE_NME']=='CENTROID CONNECTOR') | (network_speeds['out_RTE_NME']=='CENTROID CONNECTOR')]

### Put attributes on the network

In [ ]:
node_shp

In [ ]:
line_shp

In [ ]:
node_shp['X'] = node_shp.geometry.x
node_shp['Y'] = node_shp.geometry.y
nx.set_node_attributes(g, pd.Series(node_shp.X, index=node_shp.ID).to_dict(), 'X')
nx.set_node_attributes(g, pd.Series(node_shp.Y, index=node_shp.ID).to_dict(), 'Y')

### Visualizations (Slow- Due to Network Size)

In [ ]:
# plt.subplot(121)

#nx.draw(g, with_labels=True, font_weight='bold')
#plt.subplot(122)

#nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(g, nx.spring_layout(g), node_size=10, node_color='blue')
plt.show()
